<a href="https://colab.research.google.com/github/pratap977/CS6910/blob/main/CNN_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install split-folders

In [2]:
!pip install wandb

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image as im
import math as mh
from matplotlib import pyplot
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Activation,Dense,Flatten
from keras.preprocessing.image import load_img as im
from keras.preprocessing.image import save_img as sim
# import torch

In [6]:
import os
from random import shuffle
import numpy as np
import splitfolders
from PIL import Image
from torchvision import transforms
import numpy

In [7]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers,models
import wandb

In [ ]:
# for small dataset having 1000 images 
src_url = "https://storage.googleapis.com/wandb_datasets/nature_1K.zip"
src_zip = "nature_1K.zip"

In [5]:
src_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
src_zip = "nature_12K.zip"

In [6]:
%%capture
!curl -SL $src_url > $src_zip
!unzip $src_zip

In [ ]:
%%capture
!curl -SL $src_url1 > $src_zip1
!unzip $src_zip1

In [5]:
data_folders="/content/inaturalist_12K/train"

In [6]:
splitfolders.ratio(data_folders, output="outputfin", seed=1337, ratio=(.8, .2), group_prefix=None)

Copying files: 9999 files [00:36, 277.48 files/s]


In [7]:
!rm -rf nature_12K.zip

In [8]:
!rm -rf inaturalist_12K/train

In [8]:
class_labels = os.listdir("/content/outputfin/train")

In [9]:
lab={l:i for l,i in zip(class_labels,range(10))}
lab

{'Amphibia': 8,
 'Animalia': 6,
 'Arachnida': 9,
 'Aves': 7,
 'Fungi': 0,
 'Insecta': 5,
 'Mammalia': 1,
 'Mollusca': 4,
 'Plantae': 3,
 'Reptilia': 2}

In [10]:
data_augmentation  = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [11]:
imgs_tr = []
filenames = []

truth_tr = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join("/content/outputfin/train", l))
  for img in imgs_per_class:
    
    filenames.append(img.split(".")[0])
    
    img_path = os.path.join("/content/outputfin/train", l, img)
    # img = image.load_img(img_path, target_size=(224, 224))
    # img = image.img_to_array(img)
    img=Image.open(img_path)
    img = img.resize((224,224))
    if img.mode == 'L':
                continue
    normalized_image = data_augmentation(img)
    truth_tr.append(lab[l])
    # don't forget to rescale test images to match the range of inputs
    # to the network
    # img = img/255.0
    imgs_tr.append(numpy.asarray(normalized_image).transpose(1,2,0))
x_train=np.asarray(imgs_tr)
y_train=np.asarray(truth_tr)
del imgs_tr
del truth_tr

In [12]:
imgs_val = []
filenames = []

truth_val = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join("/content/outputfin/val", l))
  for img in imgs_per_class:
    
    filenames.append(img.split(".")[0])
    
    img_path = os.path.join("/content/outputfin/val", l, img)
    # img = image.load_img(img_path, target_size=(299, 299))
    # img = image.img_to_array(img)
    img=Image.open(img_path)
    img = img.resize((224,224))
    if img.mode == 'L':
                continue
    normalized_image = data_augmentation(img)
    truth_val.append(lab[l])
    # don't forget to rescale test images to match the range of inputs
    # to the network
    # img = img/255.0
    imgs_val.append(numpy.asarray(normalized_image).transpose(1,2,0))
x_val=np.asarray(imgs_val)
y_val=np.asarray(truth_val)
del imgs_val
del truth_val

In [20]:
imgs_val = []
filenames = []

truth_val = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join("/content/inaturalist_12K/val", l))
  for img in imgs_per_class:
    
    filenames.append(img.split(".")[0])
    truth_val.append(lab[l])
    img_path = os.path.join("/content/inaturalist_12K/val", l, img)
    img = image.load_img(img_path, target_size=(299, 299))
    img = image.img_to_array(img)
    # don't forget to rescale test images to match the range of inputs
    # to the network
    img = img/255.0
    imgs_val.append(img)

In [ ]:
x_train=np.asarray(imgs_tr)
y_train=np.asarray(truth_tr)
# x_val=np.asarray(imgs_val)
# y_val=np.asarray(truth_val)

In [ ]:
y_train.shape

In [ ]:
sweep_config = {
                  'method': 'grid',
                  'metric':{
                            'name':'accuracy',
                            'goal':'maximize'
                            },
                  'parameters': {
                                'filsize'   : {'values': [32, 64, 96, 128, 256]},
                                'kersize'   :{'values':[3,5,7,9]},
                                'dropout'   :{'values':[0.1,0.3,0.4]}, 
                                'denseunits':{'values':[256,100,50,60]}, 
                                'epochs'    :{'values':[5,10,20,30]},                          
                              }
              }

In [14]:
sweep_config = {
                  'method'    : "random",
                  'metric':{
                            'name':'val_accuracy',
                            'goal':'maximize'
                            },
                  'parameters': {
                                  'flt'                 : {'values' :[32, 64,16]  },
                                  'filterorg'               : {'values' :[1,2,0.5] },
                                  'kerneldim'               : {'values' :[3,4,5,7]  },
                                  'activation'              : {'values' :['sigmoid','relu','tanh','elu'] },
                                  'neurons'                 : {'values' :[20,30] },
                                  'optimizer'               : {'values' :['sgd','adam','rmsprop'] },
                                  'lr'                      : {'values' :[1e-3,1e-4,2e-3,3e-3,4e-3,3e-4] },
                                  'epochs'                  : {'values' :[5] },
                                  'batchsize'               : {'values' :[16,32,64,128] },
                                  'dropout'                 : {'values' :[0.2,0.25,0.3,0,0.1,0.5] },
                                  'batchnormalisation'       : {'values' :['YES','NO']} ,
                                }
                }

In [ ]:
sweep_config

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'accuracy'},
 'parameters': {'denseunits': {'values': [256, 100, 50, 60]},
  'dropout': {'values': [0.1, 0.3, 0.4]},
  'epochs': {'values': [5, 10, 20, 30]},
  'filsize': {'values': [32, 64, 96, 128, 256]},
  'kersize': {'values': [3, 5, 7, 9]}}}

In [13]:
def create_CNN(fil,ker,activ,batchnormalisation,dropout,neurons):
  cnn=models.Sequential()
  cnn.add(layers.Conv2D(filters=fil[0], kernel_size=(ker[0], ker[0]), activation=activ[0],input_shape=(224, 224, 3)))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())
        
        
  cnn.add(layers.Conv2D(filters=fil[1], kernel_size=(ker[1], ker[1]), activation=activ[1]))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())
                                                  
  cnn.add(layers.Conv2D(filters=fil[2], kernel_size=(ker[2], ker[2]), activation=activ[2]))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())
         
  cnn.add(layers.Conv2D(filters=fil[3], kernel_size=(ker[3], ker[3]), activation=activ[3]))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())

  cnn.add(layers.Conv2D(filters=fil[4], kernel_size=(ker[4], ker[4]), activation=activ[4]))
  cnn.add(layers.MaxPooling2D((2, 2)))
  
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())

  cnn.add(layers.Flatten())
  if(dropout!=0):
    cnn.add(layers.Dropout(dropout))
  cnn.add(layers.Dense(neurons, activation=activ[5]))
  cnn.add(layers.Dense(10, activation='softmax'))
  return cnn

In [15]:
def conlay():
  wandb.init(config=sweep_config, magic=True)  
  f= wandb.config.flt 
  fo=  wandb.config.filterorg                           
  fil=[ (int)(f*(fo)**i) if((int)(f*(fo)**i)>0) else  1 for i in range(5)]
  kd=wandb.config.kerneldim
  ker=[kd for i in range(5)]
  if(wandb.config.activation=="sigmoid"):
    ac=tf.keras.activations.sigmoid
  elif(wandb.config.activation=="tanh"):
    ac=tf.keras.activations.tanh
  elif(wandb.config.activation=="relu"):
    ac='relu'
  else:
    ac=tf.keras.activations.elu
  activ=['relu','relu','relu','relu','relu',ac]
  if(wandb.config.optimizer=="sgd"):
    optim = tf.keras.optimizers.SGD(learning_rate=wandb.config.lr)
  elif(wandb.config.optimizer=="adam"):
    optim = tf.keras.optimizers.Adam(learning_rate=wandb.config.lr)
  elif(wandb.config.optimizer=="momentum"):
    optim = tf.keras.optimizers.SGD(learning_rate=wandb.config.lr,momentum=wandb.config.gama)
  elif(wandb.config.optimizer=="rmsprop"):
    optim = tf.keras.optimizers.RMSprop(learning_rate=wandb.config.lr)
      
  cnn =create_CNN(fil,ker,activ,wandb.config.batchnormalisation,wandb.config.dropout,wandb.config.neurons)             
  cnn.compile(optimizer=optim,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  cnn.fit(x_train, y_train, batch_size=wandb.config.batchsize,epochs=wandb.config.epochs,validation_data=(x_val,y_val),callbacks=[wandb.keras.WandbCallback()])
  del cnn


In [16]:

sweep_id = wandb.sweep(sweep_config,project='A2-part-A1')

Create sweep with ID: 4zdwu10x
Sweep URL: https://wandb.ai/pratap101/A2-part-A1/sweeps/4zdwu10x


In [ ]:
wandb.agent(sweep_id,conlay,project='A2-part-A1',count=20)

wandb: Agent Starting Run: 87q90aw1 with config:
wandb: 	activation: sigmoid
wandb: 	batchnormalisation: NO
wandb: 	batchsize: 16
wandb: 	dropout: 0.1
wandb: 	epochs: 5
wandb: 	filterorg: 1
wandb: 	flt: 16
wandb: 	kerneldim: 4
wandb: 	lr: 0.003
wandb: 	neurons: 20
wandb: 	optimizer: sgd
wandb: Currently logged in as: pratap101 (use `wandb login --relogin` to force relogin)


In [14]:
def conlayss():
  # wandb.init(config=sweep_config, magic=True)    
  cnn = models.Sequential([
                     
    layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64,  kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,  kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,  kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dropout(0.2),

    
    layers.Dense(30, activation='relu'),
    layers.Dense(10, activation='softmax')
  ])
  #gc.collect()
  # heyGPU=torch.cuda.is_available()
  # if heyGPU: cnnModel=cnnModel.cuda()
  cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  cnn.fit(x_train, y_train,batch_size=64 ,epochs=10)
  del cnn
  return history
  # torch.cuda.empty_cache()

In [32]:
y_train[5]

0

In [ ]:
conlayss()

Epoch 1/10
